<a href="https://colab.research.google.com/github/recervictory/100DaysML/blob/master/09_Pandas_Data_Wrangling_Join_Combine_and_Reshape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling: Join, Combine, and Reshape

In [ ]:
# Importing
import pandas as pd
import numpy as np

## 0. Indexing with a DataFrame’s columns

In [ ]:
frame = pd.DataFrame({'roll': range(7), 
                      'marks': range(7, 0, -1), 
                      'group': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 
                      'id': [0, 1, 2, 0, 1, 2, 3]})

frame

In [ ]:
# seting new index
frameNew = frame.set_index(['group', 'id'])
frameNew

In [ ]:
# Not removing the original column
frame.set_index(['group', 'id'], drop=False)

## 1. Hierarchical Indexing
Hierarchical indexing is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. Somewhat abstractly, it provides a way for you to work with higher dimensional data in a lower dimensional form.

Hierarchical indexing plays an important role in reshaping data and **group-based** operations like forming a **pivot table**. For example, you could rearrange the data into a DataFrame using its unstack method:

In [ ]:
# Hierarchical Indexing
data =  pd.Series(np.random.randn(9), \
        index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], \
        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

In [ ]:
# Find Out Index
data.index

In [ ]:
data['b']

In [ ]:
data['b':'c']

In [ ]:
data.loc[['b', 'd']]

Hierarchical indexing plays an important role in reshaping data and group-based
operations like forming a `pivot table`. For example, you could rearrange the data into a DataFrame using its `unstack method`:

In [ ]:
data.unstack()

In [ ]:
# The inverse operation of unstack is stack
data.unstack().stack()

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), 
                     index=[['jan', 'jan', 'feb', 'feb'], [2011, 2012, 2011, 2012]],
                     columns=[['Kolkata', 'Kolkata', 'Delhi'],
                              ['Green', 'Red', 'Green']])

frame

In [ ]:
# Show index key
frame.index.names = ['month', 'year']

In [ ]:
# Show column names
frame.columns.names = ['city', 'color']

In [ ]:
frame

In [ ]:
# Reseting the index
frameNew.reset_index()

## 2. Reordering and Sorting Levels

In [ ]:
frame.swaplevel('year', 'month')

`sort_index`, on the other hand, sorts the data using only the values in a single level. When swapping levels, it’s not uncommon to also use sort_index so that the result is lexicographically sorted by the indicated level

In [ ]:
frame.sort_index(level=1)

In [ ]:
frame.swaplevel(0, 1).sort_index(level=0)

## 3. Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and `Series` have a level
option in which you can specify the level you want to `aggregate` by on a particular axis. 

In [ ]:
frame.sum(level='month')

In [ ]:
frame.mean(level='color', axis=1) # column wise

## 4. Combining and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways:
- `pandas.merge` connects rows in DataFrames based on one or more keys. This
will be familiar to users of SQL or other relational databases, as it implements
database join operations.
-  `pandas.concat` concatenates or “stacks” together objects along an axis.
-  The `combine_first` instance method enables splicing together overlapping data to fill in missing values in one object with values from another.



### Database-Style DataFrame Joins
Merge or join operations combine datasets by linking rows using one or more keys. These operations are central to relational databases (e.g., SQL-based). The merge function in pandas is the main entry point for using these algorithms on your data.

In [ ]:
# 1st Dataframe
df1 = pd.DataFrame({'name': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                    'math': range(7)})
df1

In [ ]:
# 2nd Dataframe
df2 = pd.DataFrame({'name': ['a', 'b', 'd'], 'bio': range(3)})
df2

In [ ]:
pd.merge(df1,df2)

##### Note that I didn’t specify which column to join on. If that information is not specified, merge uses the **overlapping column names** as the keys. It’s a good practice to specify explicitly, though:

In [ ]:
pd.merge(df1, df2, on='name')

In [ ]:
# If the column names are different in each object, you can specify them separately:
df3 = pd.DataFrame({'lname': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                    'math': range(7)})
df4 = pd.DataFrame({'rname': ['a', 'b', 'd'], 'bio': range(3)})
pd.merge(df3, df4, left_on='lname', right_on='rname')

### Type of JOIN in DataFrame
![join](https://cdn.mindmajix.com/blog/images/db-01_2119.png "Data Frame Join")

By default merge does an 'inner' join; the keys in the result are the intersec‐
tion, or the common set found in both tables. Other possible options are 'left',
'right', and 'outer'. The outer join takes the union of the keys, combining the
effect of applying both left and right joins:

### Different join types with how argument
- 'inner' Use only the key combinations observed in both tables
- 'left' Use all key combinations found in the left table
- 'right' Use all key combinations found in the right table
- 'output' Use all key combinations observed in both tables together



In [ ]:
pd.merge(df1, df2, how='outer')

### Merging on Index
In some cases, the merge key(s) in a DataFrame will be found in its index. In this case, you can pass left_index=True or right_index=True (or both) to indicate that the index should be used as the merge key

In [ ]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

pd.merge(left1, right1, left_on='key', right_index=True)

In [ ]:
# Using Outer Join 
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

In [ ]:
lefth = pd.DataFrame({'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'year': [2000, 2001, 2002, 2001, 2002], 
                      'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio','Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [ ]:
lefth

In [ ]:
righth

In [ ]:
pd.merge(lefth, righth, left_on=['state', 'year'], right_index=True)

In [ ]:
pd.merge(lefth, righth, left_on=['state', 'year'], right_index=True,how="outer")